In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1)
messages = [
    SystemMessage(
        content="you're a geography expert. And you only relply in English."
    ),
    
    AIMessage(
        content="Ciao, mi chiamo Paolo!"
    ),
    
    HumanMessage(
        content ="What is the distance between Mexico and Thailand. Also, what is your name?"
    )
]
chat.predict_messages(messages)


AIMessage(content='Hello! The distance between Mexico and Thailand is approximately 16,000 kilometers. My name is Paolo.')